In [1]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.utils import getDataFilesSorted

import config

In [2]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 3
TOTAL_BUDGET = 1000

In [3]:
current_team_names = {"\u0110or\u0111e Petrovi\u0107",
                      "Alisson Becker",
                      "Matty Cash",
                      "Marcos Senesi Bar\u00f3n",
                      "Adrien Truffert",
                      "Virgil van Dijk",
                      "Omar Alderete",
                      "Yankuba Minteh",
                      "Jaidon Anthony",
                      "Idrissa Gana Gueye",
                      "Alex Iwobi",
                      "Bruno Borges Fernandes",
                      "Erling Haaland",
                      "Jarrod Bowen",
                      "Dominic Calvert-Lewin",
                      }

In [4]:
# TODO: Change how files are loaded

with open(teams_filename, "r") as f:
    tempJson: list[dict] = json.load(f)

actualJson = tempJson[SELECTED_MODEL_INDEX]

In [5]:
all_player_data = pd.DataFrame.from_records(actualJson["players"])

In [6]:
print(all_player_data)

     clean_sheets  cost  expected_goals  form  gameweek  ict_index   id  \
0             4.0    57            0.00   4.0         8       14.5    1   
1             0.0    43            0.00   0.0         8        0.0    2   
2             0.0    40            0.00   0.0         8        0.0    3   
3             0.0    40            0.00   0.0         8        0.0    4   
4             4.0    62            0.48   8.2         8       27.0    5   
..            ...   ...             ...   ...       ...        ...  ...   
738           0.0    45            0.00   0.0         8        0.0  739   
739           0.0    40            0.00   0.0         8        0.0  740   
740           0.0    45            0.00   0.0         8        0.0  741   
741           0.0    45            0.00   0.0         8        0.0  742   
742           0.0    45            0.00   0.0         8        0.0  743   

                             name opposing_team  play_percent  \
0               David Raya Martín 

In [7]:
currentTeamPlayers = all_player_data.loc[all_player_data["name"].isin(current_team_names)]


for player in current_team_names:
    if(player not in currentTeamPlayers["name"].values):
        raise ValueError(f"player '{player}' not found")

In [8]:
current_team = Team.fromDataFrame(currentTeamPlayers)

In [9]:
current_team_cost = current_team.getTotalCost()
current_team_cost

945

In [10]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)

all_data = all_data[SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame.from_records(all_data["team"])

In [11]:
selected_team = Team.fromDataFrame(selected_team_df)

In [12]:
display(selected_team)

In [13]:
new_players = selected_team - current_team
print(new_players)


Total Score: 91.039

Goalkeepers:
- Nick Pope	Score: 6.08	Cost: 51	Fixture Difficulty: 0.0
- Robin Roefs	Score: 5.49	Cost: 46	Fixture Difficulty: 0.0
Defenders:
- Joško Gvardiol	Score: 6.12	Cost: 58	Fixture Difficulty: 0.0
- Jurriën Timber	Score: 6.69	Cost: 59	Fixture Difficulty: 0.0
- Jan Paul van Hecke	Score: 6.69	Cost: 44	Fixture Difficulty: 0.0
- Gabriel dos Santos Magalhães	Score: 8.6	Cost: 62	Fixture Difficulty: 0.0
Attackers:
- Nick Woltemade	Score: 5.14	Cost: 72	Fixture Difficulty: 0.0
- Igor Thiago Nascimento Rodrigues	Score: 4.83	Cost: 61	Fixture Difficulty: 0.0
Midfielders:
- Moisés Caicedo Corozo	Score: 7.39	Cost: 57	Fixture Difficulty: 0.0
- Martín Zubimendi Ibáñez	Score: 7.82	Cost: 55	Fixture Difficulty: 0.0
- Mohammed Kudus (Vice Captain) 	Score: 9.38	Cost: 67	Fixture Difficulty: 0.0
- Antoine Semenyo (Captain) 	Score: 9.52	Cost: 79	Fixture Difficulty: 0.0
- Declan Rice	Score: 7.28	Cost: 65	Fixture Difficulty: 0.0


In [14]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: 17


In [15]:
#HEURISTIC = "combined"
#MODE = SolverMode.CHEAPEST_FIRST

In [16]:
# all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(1.6307487186390148, 1.6307487186390148)

In [17]:
matrix = FixtureDifficultyMatrix()
matrix.precomputeFixtureDifficulty(0, config.CURRENT_GAMEWEEK+1, 3, config.CURRENT_SEASON, 1.0)

new_players.recalculateFixtureDifficulty(matrix)
# new_players.calculateScore(HEURISTIC)

# selected_team.recalculateFixtureDifficulty(matrix)
# selected_team.calculateScore(HEURISTIC)

# current_team.recalculateFixtureDifficulty(matrix)
# current_team.calculateScore(HEURISTIC)

In [18]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [19]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 46.591


In [20]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    # TODO: Store Team players in sorted tree instead of list
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [21]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [22]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [23]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [24]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team, pMatrix: FixtureDifficultyMatrix):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    transferData.updateFixtureDifficulties(pMatrix)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    

    newTeam.recalculateFixtureDifficulty(pMatrix)

    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            print(f"Suggested triple captain: {player.getName()}")
    return newTeam

In [25]:
newTeam = getNewTeam(current_team, selected_team, new_players, matrix)
newTeam = newTeam.toBenchTeam()
display(newTeam)
display(f"New total score: {newTeam.getTotalScore()}")

Best transfer:
Transfer from Virgil van Dijk -> Gabriel dos Santos Magalhães:
Old player: Virgil van Dijk	Score: 1.54	Cost: 61	Fixture Difficulty: 0.33333333333333337
New player: Gabriel dos Santos Magalhães	Score: 8.6	Cost: 62	Fixture Difficulty: 0.22916666666666669
- Cost change: 1
- Score change: 7.05


'New total score: 68.562'